In [1]:
from Models.PPO.PPO_Agent import PPO_Agent
import torch
import slimevolleygym
from utils import convert_to_vector, convert_to_value
import types
import slimevolleygym
import slimevolleygym.mlp as mlp
from slimevolleygym.mlp import Model
import json

In [2]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')

# Print the device as a check
print("Device used: ", DEVICE)

Device used:  mps


In [3]:
env = slimevolleygym.SlimeVolleyEnv()

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#
# Pick one of the baselines below to use as the first agent
#

# # 1a- PPO Baseline
# agent1 = PPO_Agent(12, 6, DEVICE, mlp_layers=[64, 64])
# agent1.load_models("Logging/PPO-BASELINE/20240410-091619-lr-0.0003-entcoef-0-mlp-64-kl-0.03", 1, 20003331)

# 1b- PPO Selfplay
agent1 = PPO_Agent(12, 6, DEVICE, mlp_layers=[64, 64])
agent1.load_models("Logging/PPO-SELFPLAY/20240410-171658-lr-0.0003-entcoef-0", 1, 20004667, torch.device('mps'))

# # 1c- DDQN Baseline
# buffer = PrioritizedReplayBuffer(
#         buffer_size = 1, 
#         state_dim = 1, 
#         alpha = 1, 
#         beta_init = 1, 
#         device = DEVICE
#     )
# agent1 = DDQN_Agent(state_dim = 12,
#                     action_dim = 6,
#                     hidden_layer_shape = 256,
#                     device = DEVICE,
#                     lr = 0,
#                     gamma = 0,
#                     batch_size = 0,
#                     epsilon = 0)
# agent1.load("Logging/DDQN-BASELINE/20240414-052734-lr-0.0005", 1, 10589200, buffer)
# agent1.select_action_original = agent1.select_action
# def select_action(self, state, greedy=False, deterministic=False):
#     action = self.select_action_original(state, deterministic=greedy)
#     return action, None
# agent1.select_action = types.MethodType(select_action, agent1)

# # 2- Genetic agent
# agent1 = Model(mlp.games['slimevolleylite'])
# with open('Logging/GENETIC-SELFPLAY/20240409-021844-numagents-128-totalnumgames-546000/game_546000') as f:
#     d = json.load(f)
#     agent1.set_model_params(d[0])
# def select_action(self, state, greedy=False):
#     action = self.predict(state, mean_mode=greedy)
#     action = (action > 0).astype(int) # Anything positive means a 1, 0 or negative means a 0
#     return convert_to_value(action), None
# def evaluation_mode(self):
#     pass
# agent1.select_action = types.MethodType(select_action, agent1)
# agent1.evaluation_mode = types.MethodType(evaluation_mode, agent1)

#
# Pick one of the snippets below to use as the second agent
#

# # 1a- PPO Baseline
# agent1 = PPO_Agent(12, 6, DEVICE, mlp_layers=[64, 64])
# agent1.load_models("Logging/PPO-BASELINE/20240410-091619-lr-0.0003-entcoef-0-mlp-64-kl-0.03.pt", 1, 20003331)

# 1b- PPO Selfplay
# agent2 = PPO_Agent(12, 6, DEVICE, mlp_layers=[64, 64])
# agent2.load_models("Logging/PPO-SELFPLAY/20240410-171658-lr-0.0003-entcoef-0", 1, 20004667)

# # 2- Baseline
agent2 = BaselinePolicy()
def select_action(self, state, greedy=False):
    action = self.predict(state)
    return convert_to_value(action), None
def evaluation_mode(self):
    pass
agent2.select_action = types.MethodType(select_action, agent2)
agent2.evaluation_mode = types.MethodType(evaluation_mode, agent2)

# # 3- Random agent
# agent2 = BaselinePolicy()
# def select_action(self, state, greedy=False):
#     action = convert_to_value(env.action_space.sample())
#     return action, None
# def evaluation_mode(self):
#     pass
# agent2.select_action = types.MethodType(select_action, agent2)
# agent2.evaluation_mode = types.MethodType(evaluation_mode, agent2)

# 4- Genetic agent
# agent2 = Model(mlp.games['slimevolleylite'])
# with open('Logging/GENETIC-SELFPLAY/20240409-021844-numagents-128-totalnumgames-546000/game_546000') as f:
#     d = json.load(f)
#     agent2.set_model_params(d[0])
# def select_action(self, state, greedy=False):
#     action = self.predict(state, mean_mode=greedy)
#     action = (action > 0).astype(int) # Anything positive means a 1, 0 or negative means a 0
#     return convert_to_value(action), None
# def evaluation_mode(self):
#     pass
# agent2.select_action = types.MethodType(select_action, agent2)
# agent2.evaluation_mode = types.MethodType(evaluation_mode, agent2)

# Visualize evolution of PPO self-play generations

We consider the self-play training of PPO with the entropy coefficient set to 0.1 to visualize the evolution of generations as the self-play training progresses for PPO. We chose this setting because, among the two configurations tested, it demonstrated the greatest evolution, reaching 29 generations as opposed to only 16 when the entropy coefficient was set to 0.

We analyze four distinct generations, utilizing the checkpoints that are available. We determine the specific generations by reviewing the logs displayed on Tensorboard, where we have charted the generation numbers against training steps. It is crucial to mention that we consistently save both the original training agent and its duplicate at every step during the training process. Consequently, we choose checkpoints that fall between generations X and Y, and we load agent 2 to capture the state of the agent at generation X. The generations are as follows:
- First, we load the checkpoint performed at step 5,833,042, which corresponds to the 6th generation (which is from the 5,37e+6 step). Unfortunately, this checkpoint occured just before the change to the 7th generation at the 5,84e+6 step.
- Then, we load the checkpoint performed at step 10,673,611, which corresponds to the 14th generation (which is from the 1,01e+7 step). Again, this checkpoint occured just before the change to the 15th generation at the 1,07e+7 step.
- Third, we load the checkpoint performed at step 14,738,250, which corresponds to the 22nd generation (which is from the 1,47e+7 step). Generation 23 only occurs at the 1,53e+7 step.
- Finally, we load the checkpoint performed at step 20,001,157, which corresponds to the 29th generation (which is from the 1,95e+7 step).

Thus, this code will load the geenrations 6, 14, 22 and 29, and visualize the evolution of the self-play training by playing the agents against themselves.

In [4]:
dir = "Logging/PPO-SELFPLAY/20240411-190831-lr-0.0003-entcoef-0.1"
# a) Load the self play PPO agent with entropy coefficient 0.1 at generation 6 and play it against itself
# agent1 = PPO_Agent(12, 6, DEVICE, mlp_layers=[64, 64])
# agent1.load_models(dir, 2,  5833042)
# agent2 = PPO_Agent(12, 6, DEVICE, mlp_layers=[64, 64])
# agent2.load_models(dir, 2,  5833042)

# b) Load the self play PPO agent with entropy coefficient 0.1 at generation 14 and play it against itself
# agent1 = PPO_Agent(12, 6, DEVICE, mlp_layers=[64, 64])
# agent1.load_models(dir, 2,  10673611)
# agent2 = PPO_Agent(12, 6, DEVICE, mlp_layers=[64, 64])
# agent2.load_models(dir, 2,  10673611)

# c) Load the self play PPO agent with entropy coefficient 0.1 at generation 22 and play it against itself
# agent1 = PPO_Agent(12, 6, DEVICE, mlp_layers=[64, 64])
# agent1.load_models(dir, 2,  14738250)
# agent2 = PPO_Agent(12, 6, DEVICE, mlp_layers=[64, 64])
# agent2.load_models(dir, 2,  14738250)

# d) Load the self play PPO agent with entropy coefficient 0.1 at generation 29 and play it against itself
# agent1 = PPO_Agent(12, 6, DEVICE, mlp_layers=[64, 64])
# agent1.load_models(dir, 2,  20001157)
# agent2 = PPO_Agent(12, 6, DEVICE, mlp_layers=[64, 64])
# agent2.load_models(dir, 2,  20001157)

In [5]:
# Set the agents to evaluation mode
agent1.evaluation_mode()
agent2.evaluation_mode()

# Run num_eval_episodes episodes and calculate the total return
total_return = 0
for _ in range(1):

    state1 = env.reset()
    state2 = state1
    done = False
    while not done:

        # Render the environment
        env.render(mode='human')
        
        with torch.no_grad():

            # Select the actions for each agent
            action1, _ = agent1.select_action(state1, greedy=True)
            action2, _ = agent2.select_action(state2, greedy=True)
        
        # Step the environment forward
        next_state1, reward, done, info = env.step(convert_to_vector(action1), otherAction=convert_to_vector(action2))
        next_state2 = info['otherObs']
        
        # Add the individual agents' rewards to the total returns (Since they're the same for both agents)
        total_return += reward

        # Update the states
        state1 = next_state1
        state2 = next_state2

2024-04-17 17:35:23.885 Python[79927:13394176] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/dk/m9d4fkln0d947z3d_fc1m8000000gn/T/org.python.python.savedState
